In [ ]:
#
#Lynne's email on Mar 13, 2018
# The "old" table2 is available online at https://github.com/lsst-pst/syseng_throughputs/blob/master/plots/table2
# and the "new" table2 awaiting approval (for the updates in the QE curves to match measured values) is at 
# https://github.com/lsst-pst/syseng_throughputs/blob/update-12-19-2017/plots/table2

# Bo:
# fS - Using the band weights from SRD Table 24 (I understand it is for illustration only),
# with the "old" m5 values, I get fS = 0.94
# Assuming the new throughput curves have been approved (?), I get fS = 1.01.

In [ ]:
import numpy as np

In [ ]:
# the 0th order calculation Bo has been showing at summer 2018 reviews:
m5SRD = np.array([23.9, 25.0, 24.7, 24.0, 23.3, 22.1])
#m5SRDmin = []
m5 = np.array([23.78, 24.81, 24.35, 23.92, 23.34, 22.45])
# Nv1 from SRD table 24
Nv1 = np.array([56, 80, 184, 184, 160, 160])
omega = Nv1/sum(Nv1)
fS = sum(omega*10**(0.8*(m5 - m5SRD)))
print('Current estimate: fS = %.2f'%fS)